## Import used Libraries

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from tqdm.auto import tqdm

## Get the yellow nyc taxi trips

In [ ]:
prifex = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow"
year = 2021
month = 1

In [23]:
csv_url = f'{prifex}/yellow_tripdata_{year}-{month:02d}.csv.gz'
print(csv_url)

https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2021-01.csv.gz


## Modify data types

In [ ]:
dtype = {
    "VendorID": "Int64",
    "passenger_count": "Int64",
    "trip_distance": "float64",
    "RatecodeID": "Int64",
    "store_and_fwd_flag": "string",
    "PULocationID": "Int64",
    "DOLocationID": "Int64",
    "payment_type": "Int64",
    "fare_amount": "float64",
    "extra": "float64",
    "mta_tax": "float64",
    "tip_amount": "float64",
    "tolls_amount": "float64",
    "improvement_surcharge": "float64",
    "total_amount": "float64",
    "congestion_surcharge": "float64"
}

parse_dates = [
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime"
]

df = pd.read_csv(
    csv_url,
    nrows=100,
    dtype=dtype,
    parse_dates=parse_dates
)


## Create engine to connect to postgres database 

In [ ]:
pg_user = "root"
pg_password = "root"
pg_host = "localhost"
pg_port = 5434
pg_db = "nyc_taxi"

In [ ]:
engine = create_engine(f'postgresql://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_db}')

## Get DDL schema for the database

In [ ]:
print(pd.io.sql.get_schema(df, name='yellow_taxi_data', con=engine))

## Create the table head(n=0) means the table has no data

In [ ]:
df.head(n=0).to_sql(name='yellow_taxi_data', con=engine, if_exists='replace')

## We don't want to insert all the data at once. We insert in batches and use an iterator for that

In [ ]:
df_iter = pd.read_csv(
    csv_url,
    dtype=dtype,
    parse_dates=parse_dates,
    iterator=True,
    chunksize=10000,
)
    

## Inserting data

In [ ]:
first= True
for df_chunk in tqdm(df_iter):
    if first:
        df_chunk.to_sql(name='yellow_taxi_data', con=engine, if_exists='append')
        first = False
    print(len(df_chunk))